# Discover Central Singapore -- Bubble Tea Shop


### Import libraries

In [6]:
import numpy as np 

import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json 

!pip install "geocoder == 1.38.1"
from geopy.geocoders import Nominatim
import geocoder 

import requests 
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

! pip install folium
import folium 

print("Libraries imported.")

     |████████████████████████████████| 102kB 5.4MB/s ta 0:00:011
Libraries imported.


### Get Data

In [134]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_places_in_Singapore").text

In [63]:
soup = BeautifulSoup(data, 'html.parser')

In [79]:
data={'PA':['Bishan', 'Bukit Merah', 'Bukit Timah', 'Downtown Core', 'Geylang', 'Kallang', 'Marina East', 'Marina South', 'Marina Parade', 'Museum','Newton','Novena','Orchard','Outram','Queenstown','River Valley', 'Rochor','Singapore River','Southern Islands','Straits View','Tanglin','Toa Payoh']}
df=pd.DataFrame(data,columns=['PA'])
df.head()

,PA
0,Bishan
1,Bukit Merah
2,Bukit Timah
3,Downtown Core
4,Geylang


In [65]:
df.shape

(22, 1)

### Get coordinates

In [80]:
def get_latlng(PA):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Singapore, Singapore'.format(PA))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [131]:
coords = [ get_latlng(PA) for PA in df["PA"].tolist() ]

In [68]:
coords

[[1.3507900000000745, 103.85110000000009],
 [1.2832199522478398, 103.81675993782109],
 [1.3404100000000199, 103.77221000000009],
 [1.3771599483526997, 103.95552993392594],
 [1.3114700000000425, 103.88218000000006],
 [1.3094147802789686, 103.86673041726073],
 [1.2957900000000677, 103.89544000000006],
 [1.2785700000000588, 103.85762000000005],
 [1.3025154538811112, 103.90434216102527],
 [1.3011600233709, 103.77195001500372],
 [1.312180000000069, 103.83912000000004],
 [1.3191000000000486, 103.84372000000008],
 [1.3010900000000447, 103.83965000000006],
 [1.2892410000000072, 103.83500249999999],
 [1.299660000000074, 103.80172000000005],
 [1.296855000000001, 103.83434775000002],
 [1.3041300000000433, 103.85029000000009],
 [1.2897100000000705, 103.84964000000008],
 [1.366670000000056, 103.80000000000007],
 [1.2798626945358378, 103.85359472544839],
 [1.2997505579187936, 103.81735721633365],
 [1.3344800000000419, 103.85108000000008]]

In [83]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [84]:
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [85]:
print(df.shape)
df

(22, 3)


,PA,Latitude,Longitude
0,Bishan,1.350790,103.851100
1,Bukit Merah,1.283220,103.816760
2,Bukit Timah,1.340410,103.772210
3,Downtown Core,1.377160,103.955530
4,Geylang,1.311470,103.882180
5,Kallang,1.309415,103.866730
6,Marina East,1.295790,103.895440
7,Marina South,1.278570,103.857620
8,Marina Parade,1.302515,103.904342
9,Museum,1.301160,103.771950


In [72]:
df.to_csv("df.csv", index=False)

### Create map

In [73]:
address = 'Singapore'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore 1.357107, 103.8194992.


In [129]:
map_sg = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, PA in zip(df['Latitude'], df['Longitude'], df['PA']):
    label = '{}'.format(PA)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sg)  
    
map_sg

In [94]:
map_sg.save('map_sg.html')

### Use Foursquare

In [96]:
CLIENT_ID = 'KDVNLGRWHI54PHAKNOSIZFUZUMFH4IGUT41QTAX5UTVRGNFJ' 
CLIENT_SECRET = 'SPFIV35FI5H0VXLTTHVDAU4Z3PDZKDXO41PNWAVUX4BYM3DB'
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KDVNLGRWHI54PHAKNOSIZFUZUMFH4IGUT41QTAX5UTVRGNFJ
CLIENT_SECRET:SPFIV35FI5H0VXLTTHVDAU4Z3PDZKDXO41PNWAVUX4BYM3DB


#### Get 500 venues with radius of 5km

In [99]:
radius = 5000
LIMIT = 500

venues = []

for lat, long, PA in zip(df['Latitude'], df['Longitude'], df['PA']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            PA,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [100]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PA', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2176, 7)


,PA,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Bishan,1.35079,103.8511,Bishan - Ang Mo Kio Park,1.362219,103.846250,Park
1,Bishan,1.35079,103.8511,Tori-Q,1.350549,103.848659,Japanese Restaurant
2,Bishan,1.35079,103.8511,Dian Xiao Er 店小二,1.350426,103.848988,Chinese Restaurant
3,Bishan,1.35079,103.8511,Aramsa ~ The Garden Spa,1.362292,103.847602,Spa
4,Bishan,1.35079,103.8511,Chocolat N' Spice,1.351867,103.837558,Bakery


In [101]:
venues_df.groupby(["PA"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PA,,,,,,
Bishan,100,100,100,100,100,100
Bukit Merah,100,100,100,100,100,100
Bukit Timah,100,100,100,100,100,100
Downtown Core,76,76,76,76,76,76
Geylang,100,100,100,100,100,100
Kallang,100,100,100,100,100,100
Marina East,100,100,100,100,100,100
Marina Parade,100,100,100,100,100,100
Marina South,100,100,100,100,100,100


In [102]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 191 uniques categories.


In [103]:
venues_df['VenueCategory'].unique()[:50]

array(['Park', 'Japanese Restaurant', 'Chinese Restaurant', 'Spa',
       'Bakery', 'Indian Restaurant', 'Coffee Shop', 'Café', 'Reservoir',
       'Thai Restaurant', 'Trail', 'Ice Cream Shop', 'Clothing Store',
       'Snack Place', 'Seafood Restaurant', 'Dessert Shop', 'Supermarket',
       'Pizza Place', 'BBQ Joint', 'Italian Restaurant',
       'Bubble Tea Shop', 'Hakka Restaurant', 'Pool', 'Food Court',
       'Non-Profit', 'Convenience Store', 'Hotel', 'Noodle House',
       'Cosmetics Shop', 'Flower Shop', 'College Cafeteria',
       'Nature Preserve', 'Szechuan Restaurant', 'Sushi Restaurant',
       'Pie Shop', 'Breakfast Spot', 'Asian Restaurant', 'Sandwich Place',
       'Wine Bar', 'Pet Store', 'Dog Run', 'Pub', 'Bridge',
       'Sporting Goods Shop', 'Bar', 'Vegetarian / Vegan Restaurant',
       'Whisky Bar', 'Multiplex', 'Playground', 'Scenic Lookout'],
      dtype=object)

In [104]:
"Bubble Tea Shop" in venues_df['VenueCategory'].unique()

True

### Analysis

In [107]:
sg_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

sg_onehot['PAs'] = venues_df['PA'] 

fixed_columns = [sg_onehot.columns[-1]] + list(sg_onehot.columns[:-1])
sg_onehot = sg_onehot[fixed_columns]

print(sg_onehot.shape)
sg_onehot.head()

(2176, 192)


,PAs,Accessories Store,Airport,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bay,Beach,Beer Bar,Beer Garden,Bike Trail,Bistro,Bookstore,Border Crossing,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Cantonese Restaurant,Chinese Restaurant,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Farm,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Fujian Restaurant,Furniture / Home Store,Garden,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,High School,Historic Site,History Museum,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Lounge,Malay Restaurant,Massage Studio,Mexican Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Nature Preserve,Nightclub,Non-Profit,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Recreation Center,Reservoir,Resort,Rest Area,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stables,Stadium,Supermarket,Sushi Restaurant,Szechuan Restaurant,TV Station,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Yoga Studio,Zoo,Zoo Exhibit
0,Bishan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bishan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bishan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bishan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [109]:
sg_grouped = sg_onehot.groupby(["PAs"]).mean().reset_index()

print(sg_grouped.shape)
sg_grouped

(22, 192)


,PAs,Accessories Store,Airport,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bay,Beach,Beer Bar,Beer Garden,Bike Trail,Bistro,Bookstore,Border Crossing,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Cantonese Restaurant,Chinese Restaurant,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Farm,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Fujian Restaurant,Furniture / Home Store,Garden,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,High School,Historic Site,History Museum,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Lounge,Malay Restaurant,Massage Studio,Mexican Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Nature Preserve,Nightclub,Non-Profit,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Recreation Center,Reservoir,Resort,Rest Area,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stables,Stadium,Supermarket,Sushi Restaurant,Szechuan Restaurant,TV Station,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Yoga Studio,Zoo,Zoo Exhibit
0,Bishan,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.010000,0.00,0.03,0.00,0.020000,0.01,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.01,0.000000,0.010000,0.010000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.030000,0.00,0.10,0.00,0.00,0.00,0.020000,0.00,0.00,0.030000,0.01,0.00,0.00,0.01,0.01,0.00,0.000000,0.00,0.040000,0.00,0.00,0.00,0.01,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.01,0.000000,0.040000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.000000,0.00,0.00,0.030000,0.00,0.00,0.00,0.070000,0.030000,0.00,0.000000,0.02,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.01,0.00,0.01,0.00,0.01,0.03,0.00,0.00,0.00,0.00,0.00,0.050000,0.00,0.00,0.00,0.00,0.01,0.01,0.000000,0.01,0.01,0.00,0.010000,0.01,0.000000,0.00,0.00,0.01,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.01,0.000000,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.03,0.00,0.00,0.000000,0.030000,0.00,0.00,0.01,0.00,0.00,0.040000,0.01,0.01,0.00,0.020000,0.00,0.00,0.00,0.00,0.020000,0.01,0.00,0.00,0.000000,0.000000,0.00,0.01,0.01,0.00,0.00,0.00
1,Bukit Merah,0.00,0.000000,0.000000,0.00,0.01,0.01,0.00,0.00,0.010000,0.01,0.00,0.01,0.010000,0.01,0.00,0.000000,0.00,0.01,0.000000,0.01,0.03,0.000000,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.00,0.00,0.010000,0.00,0.0

In [110]:
len(sg_grouped[sg_grouped["Bubble Tea Shop"] > 0])

16

In [111]:
sg_bts = sg_grouped[["PAs","Bubble Tea Shop"]]

In [112]:
sg_bts.head()

,PAs,Bubble Tea Shop
0,Bishan,0.010000
1,Bukit Merah,0.000000
2,Bukit Timah,0.000000
3,Downtown Core,0.013158
4,Geylang,0.010000


### Cluster 

In [113]:
kclusters = 3

kl_clustering = sg_bts.drop(["PAs"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

kmeans.labels_[0:10]

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

In [114]:
sg_merged = sg_bts.copy()

sg_merged["Cluster Labels"] = kmeans.labels_

In [115]:
sg_merged.rename(columns={"PAs": "PA"}, inplace=True)
sg_merged.head()

,PA,Bubble Tea Shop,Cluster Labels
0,Bishan,0.010000,0
1,Bukit Merah,0.000000,1
2,Bukit Timah,0.000000,1
3,Downtown Core,0.013158,0
4,Geylang,0.010000,0


In [116]:
sg_merged = sg_merged.join(df.set_index("PA"), on="PA")
print(sg_merged.shape)
sg_merged.head() 

(22, 5)


,PA,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,Bishan,0.010000,0,1.35079,103.85110
1,Bukit Merah,0.000000,1,1.28322,103.81676
2,Bukit Timah,0.000000,1,1.34041,103.77221
3,Downtown Core,0.013158,0,1.37716,103.95553
4,Geylang,0.010000,0,1.31147,103.88218


In [117]:
print(sg_merged.shape)
sg_merged.sort_values(["Cluster Labels"], inplace=True)
sg_merged

(22, 5)


,PA,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,Bishan,0.010000,0,1.350790,103.851100
17,Singapore River,0.010000,0,1.289710,103.849640
16,Rochor,0.010000,0,1.304130,103.850290
15,River Valley,0.010000,0,1.296855,103.834348
13,Outram,0.010000,0,1.289241,103.835002
12,Orchard,0.010000,0,1.301090,103.839650
11,Novena,0.010000,0,1.319100,103.843720
20,Tanglin,0.010000,0,1.299751,103.817357
10,Newton,0.010000,0,1.312180,103.839120
7,Marina Parade,0.010000,0,1.302515,103.904342


In [130]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(sg_merged['Latitude'], sg_merged['Longitude'], sg_merged['PA'], sg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [120]:
map_clusters.save('map_clusters.html')

#### Cluster 0

In [123]:
sg_merged.loc[sg_merged['Cluster Labels'] == 0]

,PA,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,Bishan,0.010000,0,1.350790,103.851100
17,Singapore River,0.010000,0,1.289710,103.849640
16,Rochor,0.010000,0,1.304130,103.850290
15,River Valley,0.010000,0,1.296855,103.834348
13,Outram,0.010000,0,1.289241,103.835002
12,Orchard,0.010000,0,1.301090,103.839650
11,Novena,0.010000,0,1.319100,103.843720
20,Tanglin,0.010000,0,1.299751,103.817357
10,Newton,0.010000,0,1.312180,103.839120
7,Marina Parade,0.010000,0,1.302515,103.904342


#### Cluster 1

In [124]:
sg_merged.loc[sg_merged['Cluster Labels'] == 1]

,PA,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
9,Museum,0.0,1,1.301160,103.771950
14,Queenstown,0.0,1,1.299660,103.801720
2,Bukit Timah,0.0,1,1.340410,103.772210
1,Bukit Merah,0.0,1,1.283220,103.816760
18,Southern Islands,0.0,1,1.366670,103.800000
19,Straits View,0.0,1,1.279863,103.853595


#### Cluster 2

In [125]:
sg_merged.loc[sg_merged['Cluster Labels'] == 2]

,PA,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
21,Toa Payoh,0.02,2,1.33448,103.85108
